In [29]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
from sklearn import preprocessing
import numpy as np
import time
import os
from PIL import Image
import json
from pathlib import Path
import numpy as np
from sklearn.preprocessing import normalize
from datetime import datetime

In [30]:
# Replace with valid values
ENDPOINT = "https://stylercv.cognitiveservices.azure.com/"
training_key = "fb709acea8b94ff9be5048b8e6c76611"
prediction_key = "b35d976bff3a4d1c9b99c7f493fc7764"
prediction_resource_id = "/subscriptions/6bb19610-9077-414c-971c-e545a2e35961/resourceGroups/styler_0405/providers/Microsoft.CognitiveServices/accounts/stylerCV-Prediction"

In [31]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [32]:
publish_iteration_name = "detectModel-test"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("Yu Project", domain_id=obj_detection_domain.id)

Creating project...


In [33]:
# Make two tags in the new project
#long_sleeve_dress_tag = trainer.create_tag(project.id, "long sleeve dress")
short_sleeve_top_tag = trainer.create_tag(project.id, "short sleeve top")
#skirt_tag = trainer.create_tag(project.id, "skirt")
shorts_tag = trainer.create_tag(project.id, "shorts")
trousers_tag = trainer.create_tag(project.id, "trousers")

In [34]:
#建一個w_json的資料夾 裡面放圖片跟json檔
json_p = 'C:/Users/qingyu/Desktop/yutest/'
jList = os.listdir(json_p)
#white_image_regions={} #這是一個dic的資料型態
category_dict = { 1 :'short_sleeve_top', 2 :'long_sleeve_top', 3 :'short_sleeve_outwear', 4 :'long_sleeve_outwear', 
                  5 :'vest', 6 :'sling', 7 :'shorts', 8 :'trousers', 9 :'skirt', 10 :'short_sleeve_dress', 
                  11 :'long_sleeve_dress', 12 :'vest_dress', 13 :'sling_dress'}
trousers_regions={}
short_sleeve_top_regions={}
shorts_regions={}
regions_arr =['short_sleeve_top_regions','trousers_regions','shorts_regions']
tag_num = 0

for file in jList:
    
    if file.endswith(".jpg"):
        img = Image.open( os.path.join(json_p,file))
        imgSize = img.size  #大小/尺寸
        pw = img.width       #图片的宽
        ph = img.height      #图片的高
        pf = img.format      #图像格式
        
# Region Left(rl) = X1 / Image Width
# Region Top(rt) = Y1 / Image Height
# Region Width(rw) = (X2 — X1) / image Width
# Region Height(rh)= (Y2 — Y1) / image Height
    
    if file.endswith(".json"):
        with open(os.path.join(json_p,file) , 'r') as reader:
            jf = json.loads(reader.read())
        strjf = str(jf)
        maxi = strjf.count('item') + 1
        for i in range (1,maxi):
            stri = str(i)
            category_id = jf['item'+stri]['category_id']
            category_name = category_dict.get(category_id)
            rl = jf['item'+stri]['bounding_box'][0]/pw
            rt = jf['item'+stri]['bounding_box'][1]/ph
            rw = (jf['item'+stri]['bounding_box'][2]-jf['item'+stri]['bounding_box'][0])/pw
            rh = (jf['item'+stri]['bounding_box'][3]-jf['item'+stri]['bounding_box'][1])/ph
            regions = category_name + '_regions'
            if regions in regions_arr:
                mydict = eval(regions)
                mydict[Path(file).stem]=[rl,rt,rw,rh,category_name]
                tag_num = tag_num + 1      

In [35]:
# Update this with the path to where you downloaded the images.
base_image_location =  'C:/Users/qingyu/Desktop/yutest/'
# Go through the data table above and create the images
print ("Adding images...")
#if file_name in short_sleeve_top_regions.keys():

def upload():
    count=0
    tagged_images_with_regions = []
    key_arr = ['short_sleeve_top_key','trousers_key','shorts_key']
    value_arr = ['short_sleeve_top_value','trousers_value','shorts_value']
    length = len(regions_arr)
    for i in range(0,length):
        key_arr[i] = list(eval(regions_arr[i]))
        value_arr[i] = list(eval(regions_arr[i]).values())
        for file_name in key_arr[i]:
            index = key_arr[i].index(file_name)
            x,y,w,h,c = value_arr[i][index]
            regions = c + '_regions'
            regions_dict = eval(regions)
            del regions_dict[file_name]
            key_arr[i] = list(regions_dict)
            value_arr[i] = list(regions_dict.values())
            tag = c + '_tag'
            regions = [ Region(tag_id=eval(tag).id, left=x,top=y,width=w,height=h) ]
            with open(base_image_location  + file_name + ".jpg", mode="rb") as image_contents:
                tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))
            count=count+1
            if(count>61):
                print("trytry")
                break;
            
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))

    tagged_images_with_regions.clear()

    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)
#end upload    

maxn = int(tag_num/64) + 1
for n in range (0,maxn): upload()  
    

Adding images...
trytry
trytry
trytry
Image batch upload failed.
Image status:  OKDuplicate
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image st

In [36]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


In [37]:
# The iteration is now trained. Publish it to the project endpoint
#trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
trainer.publish_iteration(project.id, iteration.id,publish_iteration_name ,  prediction_resource_id)

print ("Done!")

Done!


In [38]:
# Now there is a trained endpoint that can be used to make a prediction

# Open the sample image and get back the prediction results.
with open("C:/Users/qingyu/Desktop/github/Styler_test/deep_train/000414.jpg", mode="rb") as test_data:
    results = predictor.detect_image(project.id, publish_iteration_name, test_data)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))

	shorts: 94.47% bbox.left = 0.31, bbox.top = 0.04, bbox.width = 0.41, bbox.height = 0.51
	short sleeve top: 94.19% bbox.left = 0.32, bbox.top = 0.00, bbox.width = 0.38, bbox.height = 0.11
	trousers: 11.39% bbox.left = 0.38, bbox.top = 0.49, bbox.width = 0.23, bbox.height = 0.31
	trousers: 4.14% bbox.left = 0.33, bbox.top = 0.82, bbox.width = 0.20, bbox.height = 0.18
	trousers: 3.39% bbox.left = 0.39, bbox.top = 0.88, bbox.width = 0.08, bbox.height = 0.08
	trousers: 2.85% bbox.left = 0.52, bbox.top = 0.88, bbox.width = 0.08, bbox.height = 0.08
	trousers: 1.98% bbox.left = 0.46, bbox.top = 0.81, bbox.width = 0.19, bbox.height = 0.19
	shorts: 1.93% bbox.left = 0.37, bbox.top = 0.16, bbox.width = 0.26, bbox.height = 0.27
	trousers: 1.26% bbox.left = 0.38, bbox.top = 0.74, bbox.width = 0.24, bbox.height = 0.25
	trousers: 1.02% bbox.left = 0.25, bbox.top = 0.77, bbox.width = 0.63, bbox.height = 0.23
	trousers: 1.00% bbox.left = 0.03, bbox.top = 0.57, bbox.width = 0.97, bbox.height = 0.43
	sh